In [4]:
import warnings
warnings.filterwarnings("ignore")

import os
import subprocess
from scipy import constants as cst
from pint import UnitRegistry
ureg = UnitRegistry()
import sys
import shutil
import numpy as np
import multiprocessing
import matplotlib.pyplot as plt

path_to_code = "generated-codes/chapter6/"
sys.path.append(path_to_code)

from MinimizeEnergy import MinimizeEnergy
from MonteCarlo import MonteCarlo
from WriteLAMMPSfiles import write_topology_file, write_lammps_parameters, write_lammps_variables

"The values of the potential constants for argon have been taken throughout as  
E*/k= 119.76°K, r*=3.822 A, v*=23.79 cm3/mole, as determined by Michels6 from  
second virial coefficient data." [1]  

[1] Wood and Parker. The Journal of Chemical Physics, 27(3):720–733, 1957.  

In [7]:
def launch_MC_code(tau):
    kB = cst.Boltzmann*ureg.J/ureg.kelvin
    Na = cst.Avogadro/ureg.mole
    R = kB*Na
    epsilon = (119.76*ureg.kelvin*kB*Na).to(ureg.kcal/ureg.mol) # kcal/mol
    r_star = 3.822*ureg.angstrom
    sigma = r_star / 2**(1/6)
    N_atom = 300 # no units
    m_argon = 39.948*ureg.gram/ureg.mol
    T = (273.15+55)*ureg.kelvin # 55°C
    volume_star = (23.79 * ureg.centimeter**3/ureg.mole).to(ureg.angstrom**3/ureg.mole)
    cut_off = sigma*2.5
    folder = "lammps_tau"+str(tau)+"/"
    displace_mc = sigma/5 # angstrom

    volume = N_atom*volume_star*tau/Na
    box_size = volume**(1/3)

    em = MinimizeEnergy(maximum_steps=100,
                        thermo_period=10,
                        dumping_period=10,
        number_atoms=[N_atom],
        epsilon=[epsilon.magnitude], 
        sigma=[sigma.magnitude],
        atom_mass=[m_argon.magnitude],
        box_dimensions=[box_size.magnitude,
                        box_size.magnitude,
                        box_size.magnitude],
        cut_off=cut_off.magnitude,
        data_folder=folder,
    )
    em.run()

    mc = MonteCarlo(maximum_steps=200000,
        dumping_period=1000,
        thermo_period=1000,
        neighbor=50,
        displace_mc = displace_mc.magnitude,
        desired_temperature = T.magnitude,
        number_atoms=[N_atom],
        epsilon=[epsilon.magnitude], 
        sigma=[sigma.magnitude],
        atom_mass=[m_argon.magnitude],
        box_dimensions=[box_size.magnitude,
                        box_size.magnitude,
                        box_size.magnitude],
        initial_positions = em.atoms_positions*em.reference_distance,
        cut_off=cut_off.magnitude,
        data_folder=folder,
    )
    mc.run()

    write_topology_file(em, filename=folder+"initial.data")
    write_lammps_parameters(em, filename=folder+"PARM.lammps")
    write_lammps_variables(em, filename=folder+"variable.lammps")

    mycwd = os.getcwd() # initial path
    os.chdir(folder)
    shutil.copyfile("../lammps/input.lmp", ".")
    subprocess.call(["/home/simon/Softwares/lammps-2Aug2023/src/lmp_serial", "-in", "input.lmp"])
    os.chdir(mycwd)

In [8]:
if __name__ == "__main__":
    tau_values = np.round(np.logspace(-0.126, 0.882, 10),2)
    pool = multiprocessing.Pool()
    squared_numbers = pool.map(launch_MC_code, tau_values)
    pool.close()
    pool.join()

step epot maxF
0 135701194.619 1579386783.155
step epot maxFstep epot maxF

0 369910248.561 4820982372.2260 1008320826.131 14754488465.806
step epot maxF

0 2725966718.361 44870234360.490
step epot maxF
0 7382307592.382 137144932788.407
step epot maxF
0 1019397908596.964 36451495053095.000
step epot maxF
0 54168725712.782 1295433167189.838
step epot maxFstep epot maxFstep epot maxF


0 19998709184.415 420754871361.7490 384006748289.102 11957392955649.7070 146072654544.577 3984468543669.502


10 868150.450 3861120.361
10 1697961.413 6975821.094
10 220800.977 1044942.472
10 77865300.169 189200037.455
10 25774848.594 68860468.636
10 6970980.891 23021105.527
10 13067188.851 37034691.491
10 427259.876 1950378.083
20 44.260 69.537
10 3443832.504 12761963.117
20 235.068 140.009
20 -52.131 11.060
10 45283801.228 112694915.426
20 12780.893 2752.504
20 3072.193 666.689
20 1253.452 326.268
20 76778.590 10431.456
30 -125.973 1.030
30 -87.011 0.329
20 -47.968 20.188
30 324.346 250.645
30 -172.749 2

AttributeError: 'MinimizeEnergy' object has no attribute 'desired_temperature'